# Temų modeliavimas

## Kintamieji

In [1]:
dataset = 'train.csv'
column_name = 'ABSTRACT'
topics_amount = 6

## Pirminis teksto apdorojimas

In [2]:
import pandas as pd

df = pd.read_csv(dataset)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df = 0.85, stop_words = 'english')
dtm = tfidf.fit_transform(df[column_name])

## LDA

In [4]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components = topics_amount, random_state = 0)
LDA.fit(dtm)

LatentDirichletAllocation(n_components=6, random_state=0)

In [5]:
for index, topic in enumerate(LDA.components_):
    print(f'TOP 10 words for topic #{index}:')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

TOP 10 words for topic #0:
['neural', 'approach', 'method', 'models', 'based', 'networks', 'network', 'model', 'learning', 'data']


TOP 10 words for topic #1:
['model', 'density', 'mass', 'temperature', 'quantum', 'phase', 'field', 'energy', 'magnetic', 'spin']


TOP 10 words for topic #2:
['linear', 'non', 'space', 'finite', 'paper', 'functions', 'function', 'prove', 'mathbb', 'problem']


TOP 10 words for topic #3:
['pois', 'pezzo', 'ldpc', 'airy', 'operads', 'disclination', 'efimov', 'specht', 'omp', 'camassa']


TOP 10 words for topic #4:
['foliations', '_r', 'calabi', 'quaternionic', 'motives', 's_n', 'monoids', 'submanifold', 'witten', 'ricci']


TOP 10 words for topic #5:
['nodule', 'hdr', 'berezin', 'omniscience', 'multiprocessor', 'gevrey', 'u_k', 'pls', '_m', 'wr']




In [7]:
topic_results = LDA.transform(dtm)
df['Topic'] = topic_results.argmax(axis = 1)
df.head()

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,Topic
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,2
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,2
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,0
